## Part I: Imports

List the imports

In [7]:
import torch
import kagglehub
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from genetic_algorithm_mnist import perform_genetic_algorithm_mnist, quantize_model_and_evaluate_performance_mnist
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW, TrainingArguments, Trainer, set_seed
from genetic_algorithm_llm import perform_genetic_algorithm_llm, clone_model, quantize_model_and_evaluate_performance_llm

## Part II: Obtaining Models

For this task, we will experiment how our proposed quantization alogorithm performs on NN-based models compared to existing quantization algorithms. We will epxeriment with two different models: Distilled-BERT, a 66M parameters' LLM, modified for clasisifaction tasks, and a simple MNIST digit classifier with around 400k parameters.

For Distilled-BERT, we use the pre-trained model provided by Hugginface framework, and finetune it for a particular classification task. In this case, we chose a well-known simple task, proposed in one of the Kaggle competitions, of classifying tweets that are written about actual disasters, or not (https://www.kaggle.com/competitions/nlp-getting-started). We did not do any data preprocessing, and simply fetched the provided training data, which, as we will see, provided rather good results for the task, compared to submitted scores at the competition. Then, we used the trained model to inspect for further quantization tasks. 

For the MNIST classifier, we downloaded an already-trained classifier, which, in its corresponding Kaggle submission, achieved extremely high accuracy of around 99.57% (https://www.kaggle.com/models/masterhumberto/mnist-hardcore). Then, we used the trained model to inspect for further quantization tasks, and used the mnist dataset containing both easy and hard samples. 

#### Subpart I: Fine-Tuning Distilled-BERT 

First, let's load the pre-trained model from Huggin Face, with a classification head.

In [8]:

# Load a distilled-BERT model for sequence classification
model_name = "distilbert-base-uncased"
distilled_BERT_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Let's also load a corresponding tokenizer for that model (HugginFace also provides that!)

In [9]:
distilled_BERT_tokenizer = AutoTokenizer.from_pretrained(model_name)

Now, let's load the training data, which we have downloaded from the Kaggle competition. Here, we will only use the provided train data, which we will split to training, validation and datasets. Later, we can evaluate the performance of competition by submiting both the orignally-trained, and its corresponding quantized model to the comptetion, to double-check the performance of quantization. Also, we make sure that the data is reproducible, so we set the random seed.

In [10]:
full_df = pd.read_csv('train.csv')

# Extract the final test set
temp_train_texts, test_texts, temp_train_labels, test_labels = train_test_split(
    full_df['text'].tolist(),
    full_df['target'].tolist(),
    test_size=0.2,
    random_state=42
)

# Extract the training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    temp_train_texts,
    temp_train_labels,
    test_size=0.2,
    random_state=42
)

Now, let's tokenize all of the inputs.

In [11]:
train_encodings_distilled_BERT = distilled_BERT_tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
val_encodings_distilled_BERT = distilled_BERT_tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings_distilled_BERT = distilled_BERT_tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

Let's construct the datasets that are acceptable by the transformers model, for each of the training, validation and test sets.

In [12]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset_distilled_BERT = CustomDataset(train_encodings_distilled_BERT, train_labels)
val_dataset_distilled_BERT = CustomDataset(val_encodings_distilled_BERT, val_labels)
test_dataset_distilled_BERT = CustomDataset(test_encodings_distilled_BERT, test_labels)

Now, let's make sure that we only train the parameters that correspond to the classification head.

In [13]:
# Freeze all layers except the classification head
for param in distilled_BERT_model.base_model.parameters():
    param.requires_grad = False

# Only the classification head will be trained
for param in distilled_BERT_model.classifier.parameters():
    param.requires_grad = True

# Compute the number of trainable parameters in the model
num_params_trainable = sum(p.numel() for p in distilled_BERT_model.parameters() if p.requires_grad)
num_params_total = sum(p.numel() for p in distilled_BERT_model.parameters())
num_layers = len(distilled_BERT_model.state_dict().keys())

print(f"Number of trainable parameters: {num_params_trainable}")
print(f"Number of total parameters: {num_params_total}")
print(f"Number of layers (here bias for weights is considered as a layer!): {num_layers}")

Number of trainable parameters: 592130
Number of total parameters: 66955010
Number of layers (here bias for weights is considered as a layer!): 104


We can see that distilled-BERT contains 104 layers (here, we diversify between biases and their correspoinding weights, meaning that the biases of the layer can have different quantization, as in quantization tasks, the expensive operations are multiplications, so we can do that for code simplicity!). Now let's define the training parameters.

In [14]:

# Print accuracy of the validation set after the epoch
def compute_accuracy(p):
    predictions, labels = p
    preds = predictions.argmax(axis=-1)  
    return {"Accuracy": accuracy_score(labels, preds)}

# Define training arguments
training_args_distilled_BERT = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",     
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    seed=42 
)

# Initialize optimizer (AdamW is commonly used with BERT)
optimizer = AdamW(distilled_BERT_model.parameters())

# Initialize the Trainer
trainer_distilled_BERT = Trainer(
    model=distilled_BERT_model,
    args=training_args_distilled_BERT,
    train_dataset=train_dataset_distilled_BERT,
    eval_dataset=val_dataset_distilled_BERT,
    tokenizer=distilled_BERT_tokenizer,
    optimizers=(optimizer, None),
    compute_metrics=compute_accuracy
)

c:\Users\Legion Slim 7\quantization\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\Legion Slim 7\quantization\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\Legion Slim 7\AppData\Local\Temp\ipykernel_21920\3212047824.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_distilled_BERT = Trainer(


In [15]:

# Set a seed for reproducibility
set_seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)

Finally, we can train the model, and save it so we can reuse it later.

In [16]:
trainer_distilled_BERT.train()
distilled_BERT_model.save_pretrained("distilled_bert_model_final")

  0%|          | 0/6100 [00:00<?, ?it/s]c:\Users\Legion Slim 7\quantization\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:403: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  0%|          | 12/6100 [00:00<05:46, 17.56it/s]

{'loss': 0.6189, 'grad_norm': 1.324833869934082, 'learning_rate': 0.000998360655737705, 'epoch': 0.03}


  0%|          | 24/6100 [00:01<04:12, 24.06it/s]

{'loss': 0.5783, 'grad_norm': 1.0185672044754028, 'learning_rate': 0.0009967213114754098, 'epoch': 0.07}


  1%|          | 33/6100 [00:01<03:55, 25.73it/s]

...

{'loss': 0.3314, 'grad_norm': 1.5336555242538452, 'learning_rate': 1.639344262295082e-06, 'epoch': 19.97}


100%|██████████| 6100/6100 [04:36<00:00, 26.92it/s]

{'loss': 0.319, 'grad_norm': 1.3573729991912842, 'learning_rate': 0.0, 'epoch': 20.0}


                                                   
100%|██████████| 6100/6100 [04:38<00:00, 21.88it/s]


{'eval_loss': 0.41658836603164673, 'eval_Accuracy': 0.825944170771757, 'eval_runtime': 2.3192, 'eval_samples_per_second': 525.189, 'eval_steps_per_second': 33.202, 'epoch': 20.0}
{'train_runtime': 278.7309, 'train_samples_per_second': 349.585, 'train_steps_per_second': 21.885, 'train_loss': 0.40141057049641843, 'epoch': 20.0}


#### Subpart II: Preparing MNIST Classifier Model

Now, let's load both the data and model for the MNIST classifier. 

In [17]:
# Construct the validiton sets
(temp_train_images, temp_train_labels), (test_images, test_labels) = mnist.load_data()

# Construct the training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    temp_train_images,
    temp_train_labels,
    test_size=0.2,
    random_state=42
)

# Split the dataset to the training and validation sets
train_dataset_mlp_MNIST = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(64)
val_dataset_mlp_MNIST = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(64)
test_dataset_mlp_MNIST = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(64)

In [18]:
# Load the MNIST model
path_to_model = kagglehub.model_download("masterhumberto/mnist-hardcore/tensorFlow2/default") + '\\MNIST KAGGLE\\model_weights.h5'
mlp_MNIST_model = tf.keras.models.load_model(path_to_model, compile=True)

Let's see how many parameters does the model have.

In [19]:
amount_of_weights = sum(sum(np.prod(w.shape) for w in layer.get_weights()) for layer in mlp_MNIST_model.layers)
print(f"Amount of weights in the model: {amount_of_weights}")
mlp_MNIST_model.summary()

Amount of weights in the model: 392330


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,332 (1.50 MB)

 Trainable params: 391,370 (1.49 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 2 (12.00 B)

In addition to that, we can also see that the model has nine trainable layers, with whose quantization combinatios we will experiment with!

## Part III: Evaluating the Performance of Non-Quantized Models

Let's see how our models perform on the test sets. Let's begin with Distilled-BERT.

In [20]:
results = trainer_distilled_BERT.evaluate(test_dataset_distilled_BERT)
distilled_BERT_test_accuracy = results['eval_Accuracy']
distilled_BERT_test_loss = results['eval_loss']

print('Loss:', distilled_BERT_test_loss, 'Accuracy:', distilled_BERT_test_accuracy)

100%|██████████| 96/96 [00:02<00:00, 32.82it/s]

Loss: 0.4099656641483307 Accuracy: 0.8305975049244911


Now, we can do the same with the MNIST classifier model.

In [21]:
MNIST_test_loss, MNIST_test_accuracy = mlp_MNIST_model.evaluate(test_dataset_mlp_MNIST)
print(f"Loss: {MNIST_test_loss}, Accuracy: {MNIST_test_accuracy}")

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9959 - loss: 0.0132
Loss: 0.012081590481102467, Accuracy: 0.9965000152587891


In addition, having submitted both algorithms to their Kaggle comptetitions, we know that distilled-BERT achieves 80.74% accuracy, while the MNIST classifier (which was submitted not by us, but the author), achives 99.57% accuracy. 

## Part IV: Evaluating the Performance of the Baseline Algorithms

Now, we can evaluate the performance of the existing quantization algorithms on our proposed models. The reasoning and implementation overview for our chosen algorithms can be found in our report.

### Subpart I: XXX Algorithm

### Subpart II: YYY Algorithm

### Subpart III: ZZZ Algorithm

### Subpart IV: AAA Algorithm

### Part V: Evaluating the Performance of the Proposed Algorithm

Finally, we can evaluate the performance of our proposed algorithm. 

### Subpart I: GA-based Quantization on Distilled-BERT Classifier

Given our unique implementation, we can specify all of the arguments for our algorithm. For specifics of it, please take a look either at the code, or the report. Thus, let's first set the optimal parameters that we used to obtain best-possible quantization scheme. Here, we diversified it in two different cases: 
1. Priority: retaining accuracy. Obtaining the best-possible quantization, where the accuracy can almost can NOT decrease: at most 0.1% of worsening is allowed.
2. Priority: compression of the model. Obtaining the best-possible quantization, where the accuracy can decrease by maximum of 1%. 

In addition, note that here we provide the already-chosen values for the arguments, as we made all of the experiments beforehand! 

For case 1, the results we got are the following:

In [23]:
args_dBERT_case_1 = {
    'population_size': 20,
    'generations': 35,
    'mutation_rate': 0.05,
    'elitism_amount': 3,
    'crossover_top': 7,
    'bit_widths': [1, 2, 4, 8, 16, 32],
    'initial_probabilities': [0, 0.02, 0.03, 0.1, 0.2, 0.65],
    'v_q_ratio': 1,
    'min_accuracy': distilled_BERT_test_accuracy - 0.001,
    'model_name': 'distilled_bert_model_final' 
}

best_chromosome_dBERT_1, best_fitness_dBERT_1, best_accuracy_dBERT_1, best_loss_dBERT_1 = perform_genetic_algorithm_llm(distilled_BERT_model, val_dataset_distilled_BERT, args_dBERT_case_1)

Generation 1/35


100%|██████████| 77/77 [00:02<00:00, 32.48it/s]


Val-loss: 0.41658836603164673 accuracy: 0.825944170771757 quantization score: 0.3076923076923075 fitness: 0.7973473567786376
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 0 Amount of 32s: 104


100%|██████████| 77/77 [00:02<00:00, 32.83it/s]


Val-loss: 0.41688957810401917 accuracy: 0.8275862068965517 quantization score: 0.15384615384615374 fitness: 0.6109616925089614
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 104 Amount of 32s: 0


...

Val-loss: 0.4090293049812317 accuracy: 0.8300492610837439 quantization score: 0.11968380420960743 fitness: 0.5287131091908391
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 43 Amount of 16s: 46 Amount of 32s: 15


100%|██████████| 77/77 [00:02<00:00, 31.70it/s]


Val-loss: 0.40749457478523254 accuracy: 0.8275862068965517 quantization score: 0.11866646895257901 fitness: 0.5663870042966
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 1 Amount of 8s: 44 Amount of 16s: 44 Amount of 32s: 15


100%|██████████| 77/77 [00:02<00:00, 32.62it/s]

Val-loss: 0.4123367369174957 accuracy: 0.8275862068965517 quantization score: 0.12782425094104236 fitness: 0.5803869484173265
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 1 Amount of 8s: 40 Amount of 16s: 46 Amount of 32s: 17
Best fitness: 0.5287131091908391
Best accuracy: 0.8300492610837439
Corresponding validation loss: 0.4090293049812317
Corresponding chromosome: (8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 16, 16, 16, 16, 32, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 32, 32, 32, 8, 16, 32, 16, 16, 16, 8, 8, 32, 32, 8, 32, 32, 8, 32, 32, 8, 32, 8, 16, 16, 8, 32, 16, 32, 32, 8)



In [24]:
# Compute the mount of bits to encode all of the parameters
weight_dictionary = distilled_BERT_model.state_dict()

total_bits_dBERT_1 = 0
for i, key in enumerate(weight_dictionary.keys()):
    total_bits_dBERT_1 += weight_dictionary[key].numel() * best_chromosome_dBERT_1[i]

We now can evaluate the final model on the test set.

In [25]:
# Evaluate the performance of the final model on the test set
test_val_loss_dBERT_1, test_accuracy_dBERT_1 = quantize_model_and_evaluate_performance_llm(best_chromosome_dBERT_1, test_dataset_distilled_BERT, args_dBERT_case_1['model_name'])

100%|██████████| 96/96 [00:03<00:00, 31.23it/s]


In [26]:
print(f"Layers' distribution (case 1): {best_chromosome_dBERT_1}")
print(f"Final validation accuracy (case 1): {best_accuracy_dBERT_1}")
print(f"Final validation loss (case 1): {best_loss_dBERT_1}")
print(f"Final test accuracy (case 1): {test_accuracy_dBERT_1}")
print(f"Final test loss (case 1): {test_val_loss_dBERT_1}")
print(f"Total bits (case 1): {total_bits_dBERT_1}")

Layers' distribution (case 1): (8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 16, 16, 16, 16, 32, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 32, 32, 32, 8, 16, 32, 16, 16, 16, 8, 8, 32, 32, 8, 32, 32, 8, 32, 32, 8, 32, 8, 16, 16, 8, 32, 16, 32, 32, 8)
Final validation accuracy (case 1): 0.8300492610837439
Final validation loss (case 1): 0.4090293049812317
Final test accuracy (case 1): 0.8338804990151018
Final test loss (case 1): 0.402300626039505
Total bits (case 1): 833396752


Now, let's perform the same thing, but for the second case. 

In [27]:
args_dBERT_case_2 = {
    'population_size': 20,
    'generations': 35,
    'mutation_rate': 0.05,
    'elitism_amount': 3,
    'crossover_top': 7,
    'bit_widths': [1, 2, 4, 8, 16, 32],
    'initial_probabilities': [0, 0.02, 0.03, 0.1, 0.2, 0.65],
    'v_q_ratio': 0.19,
    'min_accuracy': distilled_BERT_test_accuracy - 0.01,
    'model_name': 'distilled_bert_model_final' 
}

best_chromosome_dBERT_2, best_fitness_dBERT_2, best_accuracy_dBERT_2, best_loss_dBERT_2 = perform_genetic_algorithm_llm(distilled_BERT_model, val_dataset_distilled_BERT, args_dBERT_case_2)

Generation 1/35


100%|██████████| 77/77 [00:02<00:00, 32.07it/s]


Val-loss: 0.41658836603164673 accuracy: 0.825944170771757 quantization score: 0.3076923076923075 fitness: 0.38684409723832036
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 0 Amount of 32s: 104


100%|██████████| 77/77 [00:02<00:00, 31.48it/s]


Val-loss: 0.41688957810401917 accuracy: 0.8275862068965517 quantization score: 0.15384615384615374 fitness: 0.23305517368591738
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 104 Amount of 32s: 0


...

Val-loss: 0.40791282057762146 accuracy: 0.8292282430213465 quantization score: 0.10780401071383028 fitness: 0.18530744662357834
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 1 Amount of 8s: 43 Amount of 16s: 58 Amount of 32s: 2


100%|██████████| 77/77 [00:02<00:00, 31.98it/s]


Val-loss: 0.40791282057762146 accuracy: 0.8292282430213465 quantization score: 0.10780401071383028 fitness: 0.18530744662357834
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 1 Amount of 8s: 43 Amount of 16s: 58 Amount of 32s: 2


100%|██████████| 77/77 [00:02<00:00, 32.50it/s]

Val-loss: 0.4110172688961029 accuracy: 0.8201970443349754 quantization score: 0.11594092809485051 fitness: 0.20204342097542416
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 2 Amount of 8s: 41 Amount of 16s: 57 Amount of 32s: 4
Best fitness: 0.18530744662357834
Best accuracy: 0.8292282430213465
Corresponding validation loss: 0.40791282057762146
Corresponding chromosome: (8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 32, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 8, 8, 16, 16, 16, 16, 16, 16, 16, 16, 16, 32, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 8)



In [28]:
# Compute the mount of bits to encode all of the parameters
weight_dictionary = distilled_BERT_model.state_dict()

total_bits_dBERT_2 = 0
for i, key in enumerate(weight_dictionary.keys()):
    total_bits_dBERT_2 += weight_dictionary[key].numel() * best_chromosome_dBERT_2[i]

We now can evaluate the final model on the test set.

In [29]:
# Evaluate the performance of the final model on the test set
test_val_loss_dBERT_2, test_accuracy_dBERT_2 = quantize_model_and_evaluate_performance_llm(best_chromosome_dBERT_2, test_dataset_distilled_BERT, args_dBERT_case_2['model_name'])

100%|██████████| 96/96 [00:03<00:00, 31.77it/s]


In [30]:
print(f"Layers' distribution (case 2): {best_chromosome_dBERT_2}")
print(f"Final validation accuracy (case 2): {best_accuracy_dBERT_2}")
print(f"Final validation loss (case 2): {best_loss_dBERT_2}")
print(f"Final test accuracy (case 2): {test_accuracy_dBERT_2}")
print(f"Final test loss (case 2): {test_val_loss_dBERT_2}")
print(f"Total bits (case 2): {total_bits_dBERT_2}")

Layers' distribution (case 2): (8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 32, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 8, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 8, 8, 16, 16, 16, 16, 16, 16, 16, 16, 16, 32, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 8, 16, 16, 16, 16, 8)
Final validation accuracy (case 2): 0.8292282430213465
Final validation loss (case 2): 0.40791282057762146
Final test accuracy (case 2): 0.8371634931057124
Final test loss (case 2): 0.40218308568000793
Total bits (case 2): 750673936


#### Sub-part 2: MINIST Classifier  

Last, similarly as for the distilled-BERT model, we will examine two cases for the MNIST classifier. Let's begin with case 1.

Last, let's evaluate the performance of the original model and the final quantized model on the full validation set.

In [31]:
args_MNIST_case_1 = {
    'population_size': 20,
    'generations': 25,
    'mutation_rate': 0.05,
    'elitism_amount': 3,
    'crossover_top': 7,
    'bit_widths': [1, 2, 4, 8, 16, 32],
    'v_q_ratio': 1,
    'loss_function': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    'min_accuracy': MNIST_test_accuracy - 0.001,
}

best_chromosome_MNIST_1, best_fitness_MNIST_1, best_accuracy_MNIST_1, best_loss_MNIST_1 = perform_genetic_algorithm_mnist(mlp_MNIST_model, val_dataset_mlp_MNIST, args_MNIST_case_1)

Generation 1/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Val-loss: 2.361651 accuracy: 0.0995 quantization score: 0.22222222222222224 fitness: 20.503873471154105
Amount of 1s: 0 Amount of 2s: 9 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 0 Amount of 32s: 0
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Val-loss: 2.361651 accuracy: 0.0995 quantization score: 0.4444444444444445 fitness: 20.72609569337633
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 9 Amount of 8s: 0 Amount of 16s: 0 Amount of 32s: 0
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Val-loss: 2.3140934 accuracy: 0.0995 quantization score: 0.888888888888889 fitness: 21.122982545428805
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 9 Amount of 16s: 0 Amount of 32s: 0
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Val-loss: 1.6252344 accuracy: 0.8359166666666666 quantization score: 1.777777777777778 fitness: 6.594679115083483
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 9 Amount of 32s: 0
188/1

In [32]:
# Compute the mount of bits to encode all of the parameters
quantization_layers = [layer for layer in mlp_MNIST_model.layers.copy() if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D, tf.keras.layers.BatchNormalization))]
weights_per_layer = [
    sum(np.prod(w.shape) for w in layer.get_weights())
    if layer.get_weights() else 0
    for layer in quantization_layers
]   
total_bits_MNIST_1 = sum([bit_width*n_weights for bit_width, n_weights in zip(best_chromosome_MNIST_1, weights_per_layer)])

We now can evaluate the final model on the test set.

In [33]:
# Evaluate the performance of the final model on the test set
test_val_loss_MNIST_1, test_accuracy_MNIST_1, _ = quantize_model_and_evaluate_performance_mnist(mlp_MNIST_model, best_chromosome_MNIST_1, args_MNIST_case_1['loss_function'], test_dataset_mlp_MNIST)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [34]:
print(f"Layers' distribution (case 1): {best_chromosome_MNIST_1}")
print(f"Final validation accuracy (case 1): {best_accuracy_MNIST_1}")
print(f"Final validation loss (case 1): {best_loss_MNIST_1}")
print(f"Final test accuracy (case 1): {test_accuracy_MNIST_1}")
print(f"Final test loss (case 1): {test_val_loss_MNIST_1}")
print(f"Total bits (case 1): {total_bits_MNIST_1}")

Layers' distribution (case 1): (8, 16, 8, 32, 8, 32, 4, 32, 2)
Final validation accuracy (case 1): 0.9951666666666666
Final validation loss (case 1): 1.4873466491699219
Final test accuracy (case 1): 0.9951666666666666
Final test loss (case 1): 1.4873466491699219
Total bits (case 1): 1986580


Last, analyse case 2:

In [39]:
args_MNIST_case_2 = {
    'population_size': 20,
    'generations': 25,
    'mutation_rate': 0.05,
    'elitism_amount': 3,
    'crossover_top': 7,
    'bit_widths': [1, 2, 4, 8, 16, 32],
    'v_q_ratio': 0.1,
    'loss_function': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    'min_accuracy': MNIST_test_accuracy - 0.02,
}

best_chromosome_MNIST_2, best_fitness_MNIST_2, best_accuracy_MNIST_2, best_loss_MNIST_2 = perform_genetic_algorithm_mnist(mlp_MNIST_model, val_dataset_mlp_MNIST, args_MNIST_case_2)

Generation 1/25
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Val-loss: 2.361651 accuracy: 0.0995 quantization score: 0.22222222222222224 fitness: 17.998387621773613
Amount of 1s: 0 Amount of 2s: 9 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 0 Amount of 32s: 0
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Val-loss: 2.361651 accuracy: 0.0995 quantization score: 0.4444444444444445 fitness: 18.220609843995835
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 9 Amount of 8s: 0 Amount of 16s: 0 Amount of 32s: 0
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Val-loss: 2.3140934 accuracy: 0.0995 quantization score: 0.888888888888889 fitness: 18.660298529201082
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 9 Amount of 16s: 0 Amount of 32s: 0
188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Val-loss: 1.6252344 accuracy: 0.8359166666666666 quantization score: 1.777777777777778 fitness: 4.751968186166552
Amount of 1s: 0 Amount of 2s: 0 Amount of 4s: 0 Amount of 8s: 0 Amount of 16s: 9 Amount of 32s: 0
188/

In [40]:
# Compute the mount of bits to encode all of the parameters
quantization_layers = [layer for layer in mlp_MNIST_model.layers.copy() if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D, tf.keras.layers.BatchNormalization))]
weights_per_layer = [
    sum(np.prod(w.shape) for w in layer.get_weights())
    if layer.get_weights() else 0
    for layer in quantization_layers
]   
total_bits_MNIST_2 = sum([bit_width*n_weights for bit_width, n_weights in zip(best_chromosome_MNIST_2, weights_per_layer)])

We now can evaluate the final model on the test set.

In [41]:
# Evaluate the performance of the final model on the test set
test_val_loss_MNIST_2, test_accuracy_MNIST_2, _ = quantize_model_and_evaluate_performance_mnist(mlp_MNIST_model, best_chromosome_MNIST_2, args_MNIST_case_2['loss_function'], test_dataset_mlp_MNIST)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [42]:
print(f"Layers' distribution (case 2): {best_chromosome_MNIST_2}")
print(f"Final validation accuracy (case 2): {best_accuracy_MNIST_2}")
print(f"Final validation loss (case 2): {best_loss_MNIST_2}")
print(f"Final test accuracy (case 2): {test_accuracy_MNIST_2}")
print(f"Final test loss (case 2): {test_val_loss_MNIST_2}")
print(f"Total bits (case 2): {total_bits_MNIST_2}")

Layers' distribution (case 2): (8, 16, 8, 32, 8, 32, 4, 16, 1)
Final validation accuracy (case 2): 0.9955833333333334
Final validation loss (case 2): 1.4709142446517944
Final test accuracy (case 2): 0.9955833333333334
Final test loss (case 2): 1.4709142446517944
Total bits (case 2): 1967626
